In [19]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON

In [7]:
## Get all the metagenomics datasets from research data gouv

r = requests.get('https://entrepot.recherche.data.gouv.fr/api/search?q=metagenomics&per_page=1000')
datasets = r.json()

In [17]:
## Get all the keywords that are present in the datasets

keywords = []
for item in datasets["data"]["items"]:
    if "keywords" in item:
        for keyword in item["keywords"]:
            if keyword not in keywords:
                keywords.append(keyword)

In [25]:
## Maps all the keywords with ontology terms

keywordsString = ""
keywordClassesMapping = {}
for keyword in keywords:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
    keywordsString += f""""{keyword}" """                                                                                                                                                                                                                                                                                     
sparql_query = f"""                                                                                                                                                                                                                                                                                                                 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>                                                                                                                                                                                                                                                                            
    PREFIX aro: <http://purl.obolibrary.org/obo/ARO_>                                                                                                                                                                                                                                                                               
    PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>                                                                                                                                                                                                                                                                
                                                                                                                                                                                                                                                                                                                                            
    SELECT ?class ?keywords
    FROM <http://www.ontotext.com/explicit>
    WHERE {{                                                                                                                                                                                                                                                                                                                        
        VALUES ?keywords {{                                                                                                                                                                                                                                                                                                          
            {keywordsString}                                                                                                                                                                                                                                                                                                              
        }}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
        ?class rdfs:label ?label .                                                                                                                                                                                                                                                                                                    
        FILTER (lcase(str(?label)) = lcase(?keywords))                                                                                                                                                                                                                                                                               
    }}                                                                                                                                                                                                                                                                                                                              
"""   
sparql = SPARQLWrapper("http://localhost:8081/graphdb/repositories/abromics-kg") ## Query made to the a local graph server sparql endpoint                                                                                                                                                                        
sparql.setReturnFormat(JSON)                                                                                                                                                                                                                                                                                                        
sparql.setQuery(sparql_query)                                                                                                                                                                                                                                                                                                       
try:                                                                                                                                                                                                                                                                                                                                
    res = sparql.query().convert()                                                                                                                                                                                                                                                                                                  
    recs = res["results"]["bindings"]                                                                                                                                                                                                                                                                                               
except Exception as e:                                                                                                                                                                                                                                                                                                              
    print(e)                                                                                                                                                                                                                                                                                                                        
for item in recs:                                                                                                                                                                                                                                                                                                                   
    keywordClassesMapping[item["keywords"]["value"]] = item["class"]["value"]                                                                                                                                                                                                                                                       
    print(keywordClassesMapping)

{'nutrition': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28294'}
{'nutrition': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28294', 'diet': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C15222'}
{'nutrition': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28294', 'diet': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C15222', 'metagenomic': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C201925'}
{'nutrition': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28294', 'diet': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C15222', 'metagenomic': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C201925', 'Virus': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C14283'}
{'nutrition': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28294', 'diet': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C15222', 'metagenomic': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C201925', 'Virus': 'http://ncicb.nci.nih.gov/xml/owl/EVS/T

In [33]:
## Counts the coverage of the keywords that have been found in the ontologies

mappedCount = 0
unMappedCount = 0
for keyword in keywords:
    if keyword in keywordClassesMapping.keys():
        mappedCount += 1
    else:
        unMappedCount += 1
print(f"coverage: {(mappedCount / unMappedCount)*100} %")

coverage: 29.230769230769234 %
